# 📓 01 Adquisición y Carga de Datos

Este notebook documenta el proceso de obtención de datos desde fuentes externas (OSM, IDE Chile) y su carga inicial en la base de datos PostGIS.

## 🛠️ Configuración

In [1]:
import os
import sys
from pathlib import Path
import geopandas as gpd
import osmnx as ox
from sqlalchemy import create_engine
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE RUTAS ---
project_root = Path("..")
sys.path.append(str(project_root))
load_dotenv(project_root / ".env")

# --- CONEXIÓN A POSTGIS ---
db_url = (
    f"postgresql://{os.getenv('POSTGRES_USER')}:"
    f"{os.getenv('POSTGRES_PASSWORD')}@"
    f"{os.getenv('POSTGRES_HOST', 'localhost')}:5432/"
    f"{os.getenv('POSTGRES_DB')}"
)
engine = create_engine(db_url)
print("Conexión preparada.")

## 1. Descarga de Datos

Utilizamos **OSMnx** para obtener los límites administrativos del área de estudio.

In [2]:
comuna = "San Bernardo, Chile"
print(f"Descargando límites para: {comuna}")
boundary = ox.geocode_to_gdf(comuna)
boundary

## 2. Adquisición de Edificios y Amenidades

Descargamos las capas de infraestructura y equipamiento.

In [3]:
tags = {'amenity': True, 'landuse': ['commercial', 'industrial', 'retail']}
amenities = ox.features_from_place(comuna, tags=tags)
buildings = ox.features_from_place(comuna, tags={'building': True})

print(f"Amenidades encontradas: {len(amenities)}")
print(f"Edificios encontrados: {len(buildings)}")

## 3. Carga a PostGIS

Cargamos los datos procesados al esquema `raw_data` para centralizar la información.

In [4]:
boundary.to_postgis("comuna_boundaries", engine, schema="raw_data", if_exists="replace")
print("Límites cargados exitosamente.")

amenities = amenities.apply(lambda b: b.astype(str) if isinstance(b.dtype, object) else b)
amenities.to_postgis("osm_amenities", engine, schema="raw_data", if_exists="replace")

buildings = buildings.apply(lambda b: b.astype(str) if isinstance(b.dtype, object) else b)
buildings.to_postgis("osm_buildings", engine, schema="raw_data", if_exists="replace")

print("Carga completa terminada.")

## 4. Conclusiones de Adquisición

1. **Disponibilidad:** Se logró extraer un total de +700 amenidades y +8000 edificaciones para la comuna.
2. **Estandarización:** Se resolvió la problemática de tipos de datos complejos de OSM (listas en tags) convirtiéndolos a strings para compatibilidad con SQL.
3. **Infraestructura:** La base de datos PostGIS queda poblada con las capas base necesarias para los análisis espaciales y de ML posteriores.